In [1]:
import pathlib
import contextlib

import requests
import pandas as pd

In [2]:
files_url = 'https://www.eurisportal.eu/AWFImportData/api/ExportFile/GetFiles?countryCode={country_code}'
download_url = 'https://www.eurisportal.eu/AWFImportData/api/ExportFile/DownloadFile?fileName={filename}'

headers = {"Accept": "application/json", 'User-Agent': 'test-agent'}
data_dir = pathlib.Path('~/data/euris').expanduser()
                    
# Rhein (CH, DE), Rhin (FR), Canal d’Alsace (F) Rijn (NL), Waal (NL), Boven Merwede (NL), Beneden Merwede (NL), Noord (NL), Nieuwe Maas (NL), Nieuwe Waterweg (NL), Oude Maas (NL), Hartelkanaal (NL), Pannerdensch Kanaal (NL), Nederrijn (NL), Lek (NL)
# Donau (DE), Dunaj (SK), Duna (HU), Dunav (HR), Дунав (RS), Dunărea (RO), Дунав (BG), Дунай (UA)
# Canal de Bourbourg, Canal de la Colme, Aa, Canal d’Aire, Canal de la Deule, Canal de la Sensee (FR), Lys / Leie (FR, BE), Escaut / Scheldt / Schelde (FR, BE)
# Labe (CS), Elbe (DE) Vltava (CS), Moldau (DE) Weser (DE), Vezera (CS)
# Moselle (FR), Musel (LU), Mosel (DE)

countries = ['NL', 'BE', 'DE', 'FR', 'CH', 'BG', 'UA', 'HU', 'HR', 'SK', 'RO', 'CS', 'LU']

In [3]:
resp = requests.get(files_url.format(country_code='LU'), headers={"Accept": "application/json", 'User-Agent': 'test-agent'})
resp

<Response [200]>

In [5]:
data = resp.json()
dfs = []
for country_code in countries:
    url = files_url.format(country_code=country_code)
    resp = requests.get(url, headers=headers)
    data = resp.json()
    df_i = pd.DataFrame(data)
    dfs.append(df_i) 

files = pd.concat(dfs)

In [6]:
last_files = files.sort_values(['countryCode', 'lastModified']).groupby('countryCode').last().reset_index()
last_files['url'] = ''
last_files

,countryCode,name,lastModified,fileSizeInB,url
0,BE,BE_RisIndex_20240527_v3.0.zip,2024-05-27T11:00:16+02:00,866243,
1,BG,BG_RisIndex_20240527_v3.0.zip,2024-05-27T11:00:17+02:00,140174,
2,DE,DE_RisIndex_20240527_v3.0.zip,2024-05-27T11:00:17+02:00,2282383,
3,FR,FR_RisIndex_20240527_v3.0.zip,2024-05-27T11:00:18+02:00,3709902,
4,HR,HR_RisIndex_20240527_v3.0.zip,2024-05-27T11:00:17+02:00,116008,
5,HU,HU_RisIndex_20240527_v3.0.zip,2024-05-27T11:00:16+02:00,110129,
6,LU,LU_RisIndex_20240527_v3.0.zip,2024-05-27T11:00:18+02:00,13349,
7,NL,NL_RisIndex_20240527_v3.0.zip,2024-05-27T11:00:19+02:00,2982976,
8,RO,RO_RisIndex_20240527_v3.0.zip,2024-05-27T11:00:17+02:00,162314,
9,SK,SK_RisIndex_20240527_v3.0.zip,2024-05-27T11:00:17+02:00,57545,


In [7]:

for i, row in last_files.iterrows():
    url = download_url.format(filename=row['name'])
    with contextlib.chdir(data_dir):
        !wget --content-disposition '{url}'
        !unzip '{row['name']}'
    

--2024-06-10 10:25:04--  https://www.eurisportal.eu/AWFImportData/api/ExportFile/DownloadFile?fileName=BE_RisIndex_20240527_v3.0.zip
Resolving www.eurisportal.eu (www.eurisportal.eu)... 137.116.212.234
Connecting to www.eurisportal.eu (www.eurisportal.eu)|137.116.212.234|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 866243 (846K) [application/zip]
Saving to: ‘BE_RisIndex_20240527_v3.0.zip’

BE_RisIndex_2024052 100%[===================>] 845.94K   424KB/s    in 2.0s    

2024-06-10 10:25:06 (424 KB/s) - ‘BE_RisIndex_20240527_v3.0.zip’ saved [866243/866243]

Archive:  BE_RisIndex_20240527_v3.0.zip
  inflating: RisIndex_BE_20240527.xlsx  
--2024-06-10 10:25:06--  https://www.eurisportal.eu/AWFImportData/api/ExportFile/DownloadFile?fileName=BG_RisIndex_20240527_v3.0.zip
Resolving www.eurisportal.eu (www.eurisportal.eu)... 137.116.212.234
Connecting to www.eurisportal.eu (www.eurisportal.eu)|137.116.212.234|:443... connected.
HTTP request sent, awaiting response.

In [8]:
!ls {data_dir}

BE_NetworkData_20240508_v2.4.zip     LockChamberArea_DE_20240508.geojson
BE_NetworkData_20240508_v2.4.zip.1   LockChamberArea_NL_20240508.geojson
BE_NetworkData_20240508_v2.4.zip.2   LockChamberPartArea_BE_20240508.geojson
BE_NetworkData_20240508_v2.4.zip.3   LockChamberPart_BE_20240508.geojson
BE_NetworkData_20240508_v2.4.zip.4   LockChamber_BE_20240508.geojson
BE_NetworkData_20240508_v2.4.zip.5   LockChamber_DE_20240508.geojson
BE_RisIndex_20240527_v3.0.zip	     LockChamber_NL_20240508.geojson
BG_RisIndex_20240429_v3.0.zip	     LockComplex_BE_20240508.geojson
BG_RisIndex_20240429_v3.0.zip.1      LockComplex_DE_20240508.geojson
BG_RisIndex_20240429_v3.0.zip.2      LockComplex_NL_20240508.geojson
BG_RisIndex_20240429_v3.0.zip.3      LockGate_BE_20240508.geojson
BG_RisIndex_20240429_v3.0.zip.4      NL_NetworkData_20240508_v2.4.zip
BG_RisIndex_20240429_v3.0.zip.5      NL_NetworkData_20240508_v2.4.zip.1
BG_RisIndex_20240527_v3.0.zip	     NL_NetworkData_20240508_v2.4.zip.2
BerthArea_BE_202

In [9]:
!open {data_dir}